In [41]:
import polars as pl
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv', null_values="NA")
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values="NA")
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values=["NA", "null"])
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', schema_overrides={"precip": pl.Float64, "visib": pl.Float64}, null_values="NA")

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


In [9]:
# Exercise 1: Basic Queries
# 1.1 Find all unique carriers in the airlines table
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
""")
print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


In [10]:
# 1.2 Find the top 10 destinations by number of flights
result = ctx.execute("""
SELECT dest, COUNT(*) AS flight_count
FROM flights
GROUP BY dest
ORDER BY flight_count DESC
LIMIT 10
""")
print(result)

shape: (10, 2)
┌──────┬──────────────┐
│ dest ┆ flight_count │
│ ---  ┆ ---          │
│ str  ┆ u32          │
╞══════╪══════════════╡
│ ORD  ┆ 17283        │
│ ATL  ┆ 17215        │
│ LAX  ┆ 16174        │
│ BOS  ┆ 15508        │
│ MCO  ┆ 14082        │
│ CLT  ┆ 14064        │
│ SFO  ┆ 13331        │
│ FLL  ┆ 12055        │
│ MIA  ┆ 11728        │
│ DCA  ┆ 9705         │
└──────┴──────────────┘


In [11]:
# 1.3 Find all flights that departed more than 2 hours late (120 minutes)
result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120
""")
print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

In [12]:
# Exercise 2: Aggregation
# 2.1 Calculate the average departure delay for each origin airport
result = ctx.execute("""
SELECT origin, AVG(dep_delay) AS avg_dep_delay
FROM flights
WHERE dep_delay IS NOT NULL
GROUP BY origin
ORDER BY avg_dep_delay DESC
""")
print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


In [13]:
# 2.2 Find the busiest month of the year (by number of flights)
# (They asked to peek first; keeping it as in the prompt)
peek = ctx.execute("""
SELECT *
FROM flights
LIMIT 5
""")
print(peek)

result = ctx.execute("""
SELECT month, COUNT(*) AS flights_in_month
FROM flights
GROUP BY month
ORDER BY flights_in_month DESC
LIMIT 1
""")
print(result)

shape: (5, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 517      ┆ … ┆ 1400     ┆ 5    ┆ 15     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 533      ┆ … ┆ 1416     ┆ 5    ┆ 29     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 542      ┆ … ┆ 1089     ┆ 5    ┆ 40     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 544      ┆ … ┆ 1576     ┆ 5    ┆ 45     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 554      ┆ … ┆ 762      ┆ 6    ┆ 0      ┆ 2013-01-01 11:00:00 UTC │
└──────┴───────┴─────┴──────────┴───┴──────────┴──────┴────────┴───────────

In [14]:
# 2.3 On-time performance rate for each carrier (on-time if dep_delay <= 15)
result = ctx.execute("""
SELECT
  carrier,
  AVG(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) AS on_time_rate
FROM flights
WHERE dep_delay IS NOT NULL
GROUP BY carrier
ORDER BY on_time_rate DESC
""")
print(result)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ HA      ┆ 0.929825     │
│ US      ┆ 0.878227     │
│ AS      ┆ 0.867978     │
│ AA      ┆ 0.840713     │
│ DL      ┆ 0.836812     │
│ …       ┆ …            │
│ FL      ┆ 0.733291     │
│ WN      ┆ 0.731027     │
│ F9      ┆ 0.718475     │
│ YV      ┆ 0.713761     │
│ EV      ┆ 0.695381     │
└─────────┴──────────────┘


In [15]:
# Exercise 3: Joins
# 3.1 List flights with airline names (first 20)
result = ctx.execute("""
SELECT
  f.carrier,
  a.name AS airline_name,
  f.flight,
  f.origin,
  f.dest
FROM flights f
JOIN airlines a
  ON f.carrier = a.carrier
LIMIT 20
""")
print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

In [16]:
# 3.2 Average age of planes for each carrier (age based on 2013)
result = ctx.execute("""
SELECT
  f.carrier,
  AVG(2013 - p.year) AS avg_plane_age
FROM flights f
JOIN planes p
  ON f.tailnum = p.tailnum
WHERE p.year IS NOT NULL
GROUP BY f.carrier
ORDER BY avg_plane_age DESC
""")
print(result)

shape: (16, 2)
┌─────────┬───────────────┐
│ carrier ┆ avg_plane_age │
│ ---     ┆ ---           │
│ str     ┆ f64           │
╞═════════╪═══════════════╡
│ MQ      ┆ 35.319        │
│ AA      ┆ 25.869426     │
│ DL      ┆ 16.372169     │
│ UA      ┆ 13.207691     │
│ FL      ┆ 11.385829     │
│ …       ┆ …             │
│ B6      ┆ 6.686702      │
│ F9      ┆ 4.87874       │
│ VX      ┆ 4.473643      │
│ AS      ┆ 3.33662       │
│ HA      ┆ 1.548387      │
└─────────┴───────────────┘


In [17]:
# 3.3 Flights with dep_delay > 30 and bad weather (wind_speed > 20 OR precip > 0.1)
# (Peek weather table per prompt)
wpeek = ctx.execute("""
SELECT *
FROM weather
LIMIT 5
""")
print(wpeek)

result = ctx.execute("""
SELECT
  f.year, f.month, f.day, f.dep_time, f.dep_delay, f.origin, f.dest,
  w.wind_speed, w.precip
FROM flights f
JOIN weather w
  ON f.origin = w.origin
 AND f.time_hour = w.time_hour
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1)
""")
print(result)

shape: (5, 15)
┌────────┬──────┬───────┬─────┬───┬────────┬──────────┬───────┬─────────────────────────┐
│ origin ┆ year ┆ month ┆ day ┆ … ┆ precip ┆ pressure ┆ visib ┆ time_hour               │
│ ---    ┆ ---  ┆ ---   ┆ --- ┆   ┆ ---    ┆ ---      ┆ ---   ┆ ---                     │
│ str    ┆ i64  ┆ i64   ┆ i64 ┆   ┆ f64    ┆ f64      ┆ f64   ┆ datetime[μs, UTC]       │
╞════════╪══════╪═══════╪═════╪═══╪════════╪══════════╪═══════╪═════════════════════════╡
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.0   ┆ 10.0  ┆ 2013-01-01 06:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.3   ┆ 10.0  ┆ 2013-01-01 07:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.5   ┆ 10.0  ┆ 2013-01-01 08:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.2   ┆ 10.0  ┆ 2013-01-01 09:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1011.9   ┆ 10.0  ┆ 2013-01-01 10:00:00 UTC │
└────────┴──────┴───────┴─────┴───┴────────┴──────────┴───────┴──────────────────────

In [36]:
# Exercise 4: Advanced Queries
# 4.1 Most popular aircraft types (by number of flights) – top 10
result = ctx.execute("""
SELECT
  p.manufacturer,
  p.model,
  COUNT(*) AS flight_count
FROM flights f
JOIN planes p
  ON f.tailnum = p.tailnum
GROUP BY p.manufacturer, p.model
ORDER BY flight_count DESC
LIMIT 10
""")
print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬──────────────┐
│ manufacturer                  ┆ model           ┆ flight_count │
│ ---                           ┆ ---             ┆ ---          │
│ str                           ┆ str             ┆ u32          │
╞═══════════════════════════════╪═════════════════╪══════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278        │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027        │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716        │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553        │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051        │
│ BOEING                        ┆ 737-824         ┆ 13809        │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807        │
│ BOEING                        ┆ 737-7H4         ┆ 10389        │
│ BOEING                        ┆ 757-222         ┆ 9150         │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88        

In [40]:
# Exercise 4: Advanced Queries
# 4.2 Route performance (top 10 routes by total flights) with airport names
result = ctx.execute("""
WITH routes AS (
  SELECT
    f.origin,
    ao.name AS origin_name,
    f.dest,
    ad.name AS dest_name,
    f.dep_delay
  FROM flights AS f
  JOIN airports AS ao ON f.origin = ao.faa
  JOIN airports AS ad ON f.dest   = ad.faa
)
SELECT
  origin,
  origin_name,
  dest,
  dest_name,
  COUNT(*) AS total_flights,
  AVG(dep_delay) AS avg_dep_delay,
  AVG(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) AS pct_delayed_over_30
FROM routes
GROUP BY origin, origin_name, dest, dest_name
ORDER BY total_flights DESC
LIMIT 10
""")
print(result)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_ov │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ er_30          │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11262         ┆ 8.522508      ┆ 0.098295       │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10263         ┆ 11.448621     ┆ 0.122479       │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

In [28]:
# Bonus: Compare with Polars
# Polars version of 3.3
bad_weather_delays = (
    flights.join(weather, on=["origin","time_hour"], how="inner")
    .filter(
        (pl.col("dep_delay") > 30) &
        ((pl.col("wind_speed") > 20) | (pl.col("precip") > 0.1))
    )
    .select(["year","month","day","dep_time","dep_delay","origin","dest","wind_speed","precip"])
)
print(bad_weather_delays)

shape: (4_938, 9)
┌──────┬───────┬─────┬──────────┬───┬────────┬──────┬────────────┬────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ origin ┆ dest ┆ wind_speed ┆ precip │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---    ┆ ---  ┆ ---        ┆ ---    │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ str    ┆ str  ┆ f64        ┆ f64    │
╞══════╪═══════╪═════╪══════════╪═══╪════════╪══════╪════════════╪════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 2217     ┆ … ┆ JFK    ┆ TPA  ┆ 21.86482   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 1   ┆ 2326     ┆ … ┆ JFK    ┆ LAS  ┆ 21.86482   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 1238     ┆ … ┆ EWR    ┆ JAX  ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 1400     ┆ … ┆ EWR    ┆ IAD  ┆ 24.16638   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 1455     ┆ … ┆ JFK    ┆ BOS  ┆ 20.71404   ┆ 0.0    │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …      ┆ …    ┆ …          ┆ …      │
│ 2013 ┆ 9     ┆ 22  ┆ 1540     ┆ … ┆ LGA    ┆ ATL  ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 9     ┆ 22  ┆ 1957     ┆ … ┆ LGA    ┆ PIT  ┆ 20.71404   ┆ 0.0